In [36]:
## Load necessary packages
import os
import pandas as pd
import glob
import numpy as np
import networkx as nx
# import matplotlib.pyplot as plt

## load the needed tqdm package
from tqdm import tqdm

# Enable tqdm for pandas apply
tqdm.pandas()

## Define the version number
version_number = "01_06_2026"
deployment_date = "2026-01-06"

In [37]:
## Notice!! Please change the file path of following codes into your own
raw_files_path = '/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/GtoPdb/'

## Note! define download path in the very end of the notebook!!

In [38]:
## Check all node files being read
## Read all BigGIM node csv file in group 1
import os

for f in os.listdir(raw_files_path):
    if f.endswith('.csv'):
        print(f)

approved_drug_primary_target_interactions.csv
ligands.csv
interactions.csv
GtP_to_HGNC_mapping.csv
targets_and_families.csv
approved_drug_detailed_interactions.csv
peptides.csv


In [39]:
import pandas as pd

## Read each individual csv files
## treat none as Na: , na_values=['None']
source_df = pd.read_csv(raw_files_path + 'interactions.csv')

## check the length
print("current row number is: ", len(source_df))

# ## Drop nan values
source_df = source_df.dropna(subset=['Type', 'Action'], how='all')

current row number is:  24313


/var/folders/ml/cjwyk2ps62361rr96_3b_n080000gp/T/ipykernel_79815/3618314977.py:5: DtypeWarning: Columns (6,8,9,10,11,39) have mixed types. Specify dtype option on import or set low_memory=False.
  source_df = pd.read_csv(raw_files_path + 'interactions.csv')


In [40]:
source_df.head(10)

,Target,Target ID,Target Subunit IDs,Target Gene Symbol,Target UniProt ID,Target Ensembl Gene ID,Target Ligand,Target Ligand ID,Target Ligand Subunit IDs,Target Ligand Gene Symbol,...,Affinity Low,Original Affinity Units,Original Affinity Low nm,Original Affinity Median nm,Original Affinity High nm,Original Affinity Relation,Assay Description,Receptor Site,Ligand Context,PubMed ID
0,12S-LOX,1387.0,NaN,ALOX12,P18054,ENSG00000108839,NaN,NaN,NaN,NaN,...,NaN,IC50,NaN,340.0,NaN,=,NaN,NaN,NaN,24393039
1,15-LOX-1,1388.0,NaN,ALOX15,P16050,ENSG00000161905,NaN,NaN,NaN,NaN,...,NaN,Kd,NaN,3900.0,NaN,=,Determined by surface plasmon resonance (SPR),NaN,NaN,26290290
2,15-LOX-1,1388.0,NaN,ALOX15,P16050,ENSG00000161905,NaN,NaN,NaN,NaN,...,NaN,Ki,NaN,10.0,NaN,<,NaN,NaN,NaN,20866075
3,15-LOX-1,1388.0,NaN,ALOX15,P16050,ENSG00000161905,NaN,NaN,NaN,NaN,...,NaN,IC50,NaN,200.0,NaN,=,NaN,NaN,NaN,24672829
4,15-LOX-1,1388.0,NaN,ALOX15,P16050,ENSG00000161905,NaN,NaN,NaN,NaN,...,NaN,-,NaN,NaN,NaN,=,NaN,NaN,NaN,9105693
5,15-LOX-1,1388.0,NaN,ALOX15,P16050,ENSG00000161905,NaN,NaN,NaN,NaN,...,NaN,IC50,NaN,590.0,NaN,=,NaN,NaN,NaN,NaN
6,15-LOX-2,1389.0,NaN,ALOX15B,O15296,ENSG00000179593,NaN,NaN,NaN,NaN,...,NaN,IC50,NaN,51.0,NaN,=,NaN,NaN,NaN,17656086
7,15-LOX-2,1389.0,NaN,ALOX15B,O15296,ENSG00000179593,NaN,NaN,NaN,NaN,...,NaN,Ki,NaN,2500.0,NaN,=,NaN,NaN,NaN,25111178
8,3-Mercaptopyruvate sulfurtransferase,1446.0,NaN,MPST,P25325,ENSG00000128309,NaN,NaN,NaN,NaN,...,NaN,IC50,NaN,2700.0,NaN,=,NaN,NaN,NaN,28079151
9,3-phosphoinositide dependent protein kinase 1,1519.0,NaN,PDPK1,O15530,ENSG00000140992,NaN,NaN,NaN,NaN,...,NaN,IC50,NaN,33.0,NaN,=,NaN,NaN,NaN,11896604


In [41]:
source_df[['Type', 'Action']].isna().any(axis=1).sum()

np.int64(1309)

In [42]:

## check all unique combinations between subject & object
from itertools import combinations

# Get all combinations of 3 column names
col_combos = ['Type', 'Action']

# Get unique combinations and their counts
# combo_counts = source_df.groupby(col_combos).size().reset_index(name='count')

combo_counts = (
    source_df
    .groupby(col_combos, dropna=False)
    .size()
    .reset_index(name='count')
)

print(combo_counts)

                Type           Action  count
0          Activator       Activation    289
1          Activator          Agonist    176
2          Activator          Binding      6
3          Activator     Full agonist     29
4          Activator  Partial agonist     23
..               ...              ...    ...
72  Subunit-specific     Potentiation      6
73               NaN          Binding    220
74               NaN      Competitive      3
75               NaN       Inhibition      6
76               NaN     Potentiation      1

[77 rows x 3 columns]


In [43]:
## Define the output path for node & edge files after formatting
download_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/temp/gtopdb_source_combos.tsv'

## download both node and edge files
## Download the result df
## disable download for testing
combo_counts.to_csv(download_path_node_file, sep ='\t', index=False)